Постановка задачи:

Предполагаю сравнивать схожесть научных интересов путём анализа названий статей, названий конференций, в которых участвует преподаватель, а также курсовых и научных работ студентов и аспирантов. По результатам анализа можно рекомендовать научных руководителей и собирать научные группы, специализирующиеся на каком-то определённом направлении.
Сначала хочу сделать модель для технических специальностей, потом масштабировать его до всех наук. При масштабировании можно для первоначальной сортировки использовать классификатор УДК (он во всех статьях обычно указывается).
Данные можно собирать с сайта https://www.elibrary.ru/, но там стоит защита от автоматизированного сбора информации и они блокируют пользователя. Поэтому либо ручной сбор, либо ещё можно вот этим сайтом пользоваться: https://cyberleninka.ru/. Но киберленинка хуже, т.к там нет структурированной информации об авторе, так что возможно для неё ещё можно сделать модель для анализа текста статей и извлечения информации об авторах.
Про защиту киберленинки от автоматизированного сбора информации ничего не знаю.

Мотивация этой работы это будущая защита диссертации по научной специальности 2.3.8 “Информатика и информационные процессы”, возможные направления исследования, которые предполагается закрыть этой темой (для справки просто пишу, на них можно не сосредотачиваться):

- Разработка компьютерных методов и моделей описания, оценки и оптимизации информационных процессов и ресурсов, а также средств анализа и выявления закономерностей на основе обмена информацией пользователями и возможностей используемого программно-аппаратного обеспечения.
- Обеспечение информационных систем и процессов, применения информационных технологий и систем в принятии решений на различных уровнях управления. Общие принципы и основы организации информационных служб и электронных библиотек.
- Разработка методов обработки, группировки и аннотирования информации, в том числе, извлеченной из сети интернет, для систем поддержки принятия решений, интеллектуального поиска, анализа.

### Теоретические задачи:
1. Провести анализ того что уже сделано в этом направлении. Что можно рассмотреть:
 - методы кластеризации текстов
 - извлечение информации (например, поиск удк по шаблону)
 - рекомендательные системы на схожести интересов
2. Посмотреть научные статьи по теме
3. Построить модель работы будущей системы

4. Старые статьи выгодно использовать, т.к их делали без использования ИИ, использование которого снижает качество новых моделей машинного обучения (найти подтверждение, написать своё, можно вообще сравнить качество работы модели на старых работах и на новых?)

### Практические задачи:
1. <font color="green">Собрать данные:</font> 13.01

 - <font color="gray"> искусственно собрать 50 научных работ с авторами, структурировать всё, можно попытаться автоматически ткнуться в elibrary, но чую забанят меня. надо может обходы посмотреть, но это уже какая-то кибербезопасность пошла, я её не умею</font>
 - <font color="gray">сделать автоматическое распознавание текста</font>
 - <font color="green">сделать веб-скрапинг для киберленинки</font>
 - ? собрать названия дипломных работ
 - ? собрать связку преподаватель - какими работами руководил
2. Обработать данные в приемлемый вид
 - лемматизация, очистка от стоп слов 14.01
 - посмотреть статистики, схожести текстов и любых других параметров которые на этом этапе я откопаю (TF-IDF)
 - извлечь полезную информации об авторе, учреждении, основных мыслей статьи (можно построить облако слов, облако близких тем, как в последней недоделанной домашке) 15.01
 - векторизовать названия статей, текстов, и всего остального текста 17.01
 - можно проверять минимальную длину слов в статье, а то попадается всякое... 17.01
 - <font color="green">собрать всё в датафрейм</font>
3. Классическое машинное обучение
 - задача регрессии: вводим название конференции, и высвечивается вероятность с которой данный человек будет в ней участвовать. Можно тут линейные модели обкатать, логистическую регрессию например.
 - задача классификации: группируем людей по научным областям на основе названий их работ, здесь два варианта, широкая кластеризация (по фундаментальным наукам) и узкая (внутри одной науки). Обучить ставить удк?
 - кластеризация, пытаемся выделить людей со схожими научными интересами без изначально заданных областей знаний, что потом ляжет в основу рекомендаций для научных групп
 - меряем качество этого всего измеряем (все метрики забыла уже мда)
 - ? автоматическая оценка эффективности работы научного сотрудника, ранжирование (для начала по количеству написанных работ, но можно ещё оценивать "серьёзность" проведённого исследования, над мерилом "серьёзности" надо подумать, хотя звучит как антиплагиат номер два)
4. Рекомендательные системы
 - вводим id (или ФИО) преподавателя, и высвечиваются рекомендуемые ему конференции и научные журналы, тип "user" 
 - вводим название конференеции, и высвечиваются юзеры, которым это было бы интересно, тип "item"
 - ? вводим тему диплома человека, высвечиваются по убыванию потенциальные дипломные руководители

## ПАРСИМ КИБЕРЛЕНИНКУ

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# метод get
res = requests.get('https://cyberleninka.ru/article/n/printsipy-proektirovaniya-integralnoy-modeli-otsenki-nadezhnosti-informatsionno-vychislitelnyh-sistem')
res
# res.status_code

<Response [200]>

In [3]:
soup = BeautifulSoup(res.text)

In [4]:
res = requests.get('https://cyberleninka.ru/article/c/computer-and-information-sciences')
soup = BeautifulSoup(res.text)
# функция find_all позволяет найти все указанные тег с нужными атрибутами (с вложениями), возвращает список
articles = soup.find_all('li')
print(len(articles))
print(articles[0])

26
<li>
<a href="/article/n/printsipy-proektirovaniya-integralnoy-modeli-otsenki-nadezhnosti-informatsionno-vychislitelnyh-sistem"><div class="title">Принципы проектирования интегральной модели оценки надежности информационно-вычислительных систем</div>
<p>В данной статье проведен краткий анализ существующих моделей определения надежности ИВС, а так же представлен один их походов используемых при проектировании интегральной модели оценки надежности информационновычислительных систем, которая бы учитывала как алгоритмические характеристики...</p>
<span>2008 / Гузик В. Ф., Самойленко А. П., Мунтян Е. Р.</span>
<div class="labels">
<div class="label vak">ВАК</div>
<div class="label-cc"><img class="black" src="/images/tsvg/cc-label.svg"/></div></div>
</a>
</li>


In [5]:
try: #все статьи с первой страницы
    for el in articles:
        title = el.find('div', 'title').text
        print('Заголовок:',title)
        annotation = el.find('p').text
        print('Аннотация:',annotation)
        date = el.find('span').text[0:4]
        print('Год выхода:', date)
        authors = el.find('span').text[7:]
        print('Авторы:', authors)
        article_level = el.find('div', 'labels').get_text(separator=' ').strip() #сепаратором делаю пробел между словами, стрипом удаляю пробелы в начале и конце
        print('Уровень статьи:', article_level)
        link = el.find('a').get('href') #сепаратором делаю пробел между словами, стрипом удаляю пробелы в начале и конце
        print('Ссылка:', link)
        print()
except AttributeError:
    pass

Заголовок: Принципы проектирования интегральной модели оценки надежности информационно-вычислительных систем
Аннотация: В данной статье проведен краткий анализ существующих моделей определения надежности ИВС, а так же представлен один их походов используемых при проектировании интегральной модели оценки надежности информационновычислительных систем, которая бы учитывала как алгоритмические характеристики...
Год выхода: 2008
Авторы: Гузик В. Ф., Самойленко А. П., Мунтян Е. Р.
Уровень статьи: ВАК
Ссылка: /article/n/printsipy-proektirovaniya-integralnoy-modeli-otsenki-nadezhnosti-informatsionno-vychislitelnyh-sistem

Заголовок: Алгоритм взаимного исключения в пиринговых системах
Аннотация: Предложен алгоритм взаимного исключения одновременного доступа разных процессов к одному и тому же объекту в динамических П2П системах, ориентированный на уменьшение служебного трафика. Основная идея его передача сообщений между запросными узлами и координатором. Информация n дубликатов объекта...
Год в

Напишем функцию на основе всех предыдущих действий, которая будет возвращать датафрейм с датой, заголовком, категорией и ссылкой на полный текст поста

In [9]:
#сначала я собирала данные с первых 120 страниц, но после этапа выделения статистических характеристик БД поняла,
#что выборка нерепрезентатитвная получается. попробую через рандом собрать
import random

random_page = []
for i in range(1,120):
    random_number = random.randint(1, 4232)
    random_page.append(random_number)
print(random_page)

[2664, 1334, 3, 1461, 1661, 3420, 1026, 1912, 3415, 2550, 4153, 87, 550, 1652, 2346, 155, 3078, 1073, 2018, 3180, 1406, 701, 3119, 2516, 1517, 3284, 319, 1219, 4152, 3463, 1995, 4212, 1623, 2529, 1360, 1796, 3396, 2733, 3981, 1366, 2352, 1266, 3524, 1313, 628, 4056, 3226, 32, 1930, 2211, 1310, 719, 3980, 2731, 307, 1615, 2898, 784, 147, 2293, 4164, 2987, 4128, 2292, 1810, 1640, 2521, 2977, 1946, 1982, 4152, 942, 2616, 1605, 573, 414, 3636, 2004, 3129, 3905, 795, 3452, 2338, 1428, 1010, 724, 3650, 1161, 1973, 1310, 3820, 3655, 3847, 2960, 2386, 2455, 2123, 910, 1188, 3137, 4192, 2012, 444, 3904, 2880, 3738, 75, 3554, 3404, 2381, 999, 3927, 4062, 3161, 4207, 2273, 1536, 380, 4032]


In [11]:
for page in random_page[0:3]:
    print(page) #три страницы вытаскиваем чтоб посмотреть
    url = f'https://cyberleninka.ru/article/c/computer-and-information-sciences/{page}'
    req = requests.get(url).text
    soup = BeautifulSoup(req)
 #   print(soup) тут можно посмотреть экземпляр объекта супа

2664
1334
3


In [14]:
kiber_dateframe = pd.DataFrame() #временный датафрейм для функции, в который добавляется новая информация с сайта
articles_df = pd.read_excel('date_random.xlsx')

In [15]:
articles_df

""


In [24]:
def get_kiberleninka(dateframe, pages): #извлекаем информацию с каждой случайной. Чтобы продолжить, меняй range(112+1, страница на которой остановиться)
    for page in pages:
  #      print(page)
        url = f'https://cyberleninka.ru/article/c/computer-and-information-sciences/{page}'
        req = requests.get(url).text
        soup = BeautifulSoup(req)

        articles = soup.find_all('li')

        for el in articles:
            try:
                title = el.find('div', 'title').text
                annotation = el.find('p').text
                date = el.find('span').text[0:4]
                authors = el.find('span').text[7:]
                article_level = el.find('div', 'labels').get_text(separator=' ').strip() #сепаратором делаю пробел между словами, стрипом удаляю пробелы в начале и конце
                link = el.find('a').get('href') 
            except AttributeError:
                break
            
            row = {'date': date, 'title': title, 'link': link, 'annotation':annotation, 'authors':authors, 
                  'article_level':article_level}
            dateframe = pd.concat([dateframe, pd.DataFrame([row])]) 
    return dateframe

articles_df = pd.concat([articles_df, get_kiberleninka(kiber_dateframe, random_page[11:120])]) #чтобы набрать данных здесь надо менять срез, это номера страниц на киберленинке
articles_df.reset_index(drop=True)

,date,title,link,annotation,authors,article_level
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,
...,...,...,...,...,...,...
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",ВАК
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",ВАК
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",ВАК
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,ВАК


In [25]:
articles_df.to_excel('date_random.xlsx', index=False) #сохранение в файл (не перезапиши что есть!)

In [26]:
articles_df.reset_index(drop=True)

,date,title,link,annotation,authors,article_level
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,
...,...,...,...,...,...,...
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",ВАК
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",ВАК
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",ВАК
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,ВАК


In [147]:
articles_df = pd.read_excel('date_random.xlsx')

In [188]:
articles_df.iloc[1890]['link'] #периодически просит капчу, тут можно удобно смотреть где

'/article/n/k-voprosu-o-kreativnosti-v-neyrosetyah-iskusstvennogo-intellekta'

In [150]:
articles_df['link'][0:10]

0    /article/n/problema-obrabotki-dannyh-georadiol...
1    /article/n/understanding-and-designing-modern-...
2    /article/n/razrabotka-autentifikatsionnoi-mode...
3    /article/n/razrabotka-modeli-informatsionnyh-p...
4    /article/n/avtomatizirovannaya-sistema-upravle...
5    /article/n/sredstva-analiza-i-vizualizatsii-me...
6    /article/n/analiz-blokchei-n-platform-dlya-rea...
7    /article/n/metodika-snizheniya-tributov-v-mash...
8    /article/n/primenenie-geoinformatsionnyh-siste...
9    /article/n/balalar-zhasynda-y-ua-ytsha-zh-ne-t...
Name: link, dtype: object

In [194]:
#функция для добавления текста статей в датафрейм
def add_full_text(posts_df):
    i = 2100
    for el in articles_df['link'][2100:2340]:
        print(i)
        el = 'https://cyberleninka.ru'+el
        req = requests.get(el).text
        soup = BeautifulSoup(req)
        time.sleep(0.3)
        full_text = soup.find('div', class_='ocr').text.strip()
        posts_df.loc[i, 'text'] = full_text
        i += 1
        if i == 2340:
            break
    return posts_df

add_full_text(articles_df)

#чтобы добавить новый блок информации надо поменять i в начале на +1 к тому что заполнено, и в конце сделать i==индекс на котором остановиться
#и изменить срез (включительно, или пустые строки будут)

2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299


,date,title,link,annotation,authors,article_level,text
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,NaN,﻿GEOLOGICAL AND MINERALOGICAL SCIENCES\nTHE PR...
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",NaN,﻿TECHNICAL SCIENCES\nUNDERSTANDING AND DESIGNI...
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",NaN,﻿DEVELOPMENT OF AUTHENTICATION MODEL FOR CLOUD...
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,NaN,﻿DEVELOPMENT OF A MODEL OF INFORMATION FLOWS I...
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,NaN,﻿AUTOMATED CONTROL SYSTEM OF THE SOLAR AIR MAN...
...,...,...,...,...,...,...,...
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",ВАК,﻿Модель пилота как средство валидации перспект...
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",ВАК,﻿Сравнительный анализ платформ для цифровизаци...
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",ВАК,﻿Программный комплекс распределенного тестиров...
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,ВАК,﻿í rpQm'ÓTQ\nISSN 2782-4543 (online) ISSN 1997...


In [195]:
articles_df.reset_index(drop=True) #надо обновлять индекс перед следующей итерацией выкачивания статей

,date,title,link,annotation,authors,article_level,text
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,NaN,﻿GEOLOGICAL AND MINERALOGICAL SCIENCES\nTHE PR...
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",NaN,﻿TECHNICAL SCIENCES\nUNDERSTANDING AND DESIGNI...
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",NaN,﻿DEVELOPMENT OF AUTHENTICATION MODEL FOR CLOUD...
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,NaN,﻿DEVELOPMENT OF A MODEL OF INFORMATION FLOWS I...
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,NaN,﻿AUTOMATED CONTROL SYSTEM OF THE SOLAR AIR MAN...
...,...,...,...,...,...,...,...
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",ВАК,﻿Модель пилота как средство валидации перспект...
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",ВАК,﻿Сравнительный анализ платформ для цифровизаци...
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",ВАК,﻿Программный комплекс распределенного тестиров...
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,ВАК,﻿í rpQm'ÓTQ\nISSN 2782-4543 (online) ISSN 1997...


In [196]:
articles_df.to_excel('date_random.xlsx', index=False) #сохранение в файл (не перезапиши что есть!)